# WeRateDogs Project- Wrangling & Analyzing Twitter Data

- Philipp Keupp

- Dezember 2018

- [Introduction](#intro)
- [Gathering Data](#gath)
- [Assessing Data](#ass)
    - [Quality and Tidiness Issues](#que)
- [Cleaning Data](#cle)

## <a id="intro">Introduction</a>

The goal of this project is to wrangle the WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The challenge lies in the fact that the Twitter archive is great, but it only contains very basic tweet information that comes in JSON format. For a successful project, I needed to gather, asses and clean the Twitter data for a worthy analysis and visualization.

In [1]:
# import main libraries

import pandas as pd
import numpy as np
import os
import requests as rq
import json
import time
import tweepy

## <a id="gath">Gathering Data</a>

In [2]:
# load data
data_twitter = pd.read_csv('twitter-archive-enhanced.csv')
data_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [3]:
data_twitter.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [15]:
# Download tsv file
folder_name = 'images_prediction'
# Make directory if it doesn't already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = rq.get(url)
r

with open(os.path.join(folder_name,
                      url.split('/')[-1]), mode='wb') as file:
    file.write(r.content)

In [4]:
#open tsv file
images = pd.read_table('images_prediction/image-predictions.tsv',
                       sep='\t')

In [ ]:
#

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

In [ ]:

import json
tweet_ids = data_twitter.tweet_id.values

with open('tweet_json.txt', 'a', encoding='utf8') as outfile:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except Exception as e:
            print(tweet_id,e) 

In [5]:
data_list = []
data_dict = {
                'tweet_id': '',
                'favorite_count': '',
                'retweet_count': '',
            }
with open('tweet_json.txt') as json_file:
    for line in json_file:
        data = json.loads(line)
        data_dict['tweet_id'] = data['id']
        data_dict['favorite_count'] = data['favorite_count']
        data_dict['retweet_count'] = data['retweet_count']
        data_list.append(data_dict.copy())
favorite_retweet_table = pd.DataFrame(data_list)

## <a id="ass">Assessing Data</a>

In [6]:
data_twitter.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [7]:
data_twitter.tail(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2346,666058600524156928,NaN,NaN,2015-11-16 01:01:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is the Rand Paul of retrievers folks! He'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666058600...,8,10,the,None,None,None,None
2347,666057090499244032,NaN,NaN,2015-11-16 00:55:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",My oh my. This is a rare blond Canadian terrie...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666057090...,9,10,a,None,None,None,None
2348,666055525042405380,NaN,NaN,2015-11-16 00:49:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a Siberian heavily armored polar bear ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666055525...,10,10,a,None,None,None,None
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None
2350,666050758794694657,NaN,NaN,2015-11-16 00:30:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a truly beautiful English Wilson Staff...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758...,10,10,a,None,None,None,None
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [8]:
data_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
data_twitter.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [10]:
images.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [11]:
images.tail(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2065,890240255349198849,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,1,Pembroke,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True
2066,890609185150312448,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
2067,890729181411237888,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
2068,890971913173991426,https://pbs.twimg.com/media/DF1eOmZXUAALUcq.jpg,1,Appenzeller,0.341703,True,Border_collie,0.199287,True,ice_lolly,0.193548,False
2069,891087950875897856,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [12]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
images.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [14]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [15]:
sum(images.jpg_url.duplicated())

66

In [16]:
print(images.p1_dog.value_counts())
print(images.p2_dog.value_counts())
print(images.p3_dog.value_counts())

True     1532
False     543
Name: p1_dog, dtype: int64
True     1553
False     522
Name: p2_dog, dtype: int64
True     1499
False     576
Name: p3_dog, dtype: int64


In [17]:
favorite_retweet_table.head(10)

,favorite_count,retweet_count,tweet_id
0,38197,8366,892420643555336193
1,32748,6177,892177421306343426
2,24664,4089,891815181378084864
3,41522,8503,891689557279858688
4,39719,9207,891327558926688256
5,19936,3064,891087950875897856
6,11665,2029,890971913173991426
7,64457,18568,890729181411237888
8,27395,4205,890609185150312448
9,31434,7270,890240255349198849


In [18]:
favorite_retweet_table.tail(10)

,favorite_count,retweet_count,tweet_id
2332,111,57,666058600524156928
2333,294,141,666057090499244032
2334,431,244,666055525042405380
2335,1210,841,666051853826850816
2336,132,59,666050758794694657
2337,107,40,666049248165822465
2338,296,139,666044226329800704
2339,125,44,666033412701032449
2340,129,47,666029285002620928
2341,2541,508,666020888022790149


In [19]:
favorite_retweet_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
favorite_count    2342 non-null int64
retweet_count     2342 non-null int64
tweet_id          2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


In [20]:
favorite_retweet_table.describe()

,favorite_count,retweet_count,tweet_id
count,2342.000000,2342.000000,2.342000e+03
mean,7998.083689,2949.687020,7.422212e+17
std,12379.215144,4960.763537,6.832408e+16
min,0.000000,0.000000,6.660209e+17
25%,1382.750000,592.500000,6.783509e+17
50%,3482.500000,1376.500000,7.186224e+17
75%,9803.500000,3441.250000,7.986971e+17
max,164902.000000,84054.000000,8.924206e+17


In [21]:
data_twitter['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [22]:
print(data_twitter.loc[data_twitter.rating_numerator == 204, 'text']) 
print(data_twitter.loc[data_twitter.rating_numerator == 143, 'text']) 
print(data_twitter.loc[data_twitter.rating_numerator == 666, 'text']) 
print(data_twitter.loc[data_twitter.rating_numerator == 1176, 'text'])
print(data_twitter.loc[data_twitter.rating_numerator == 144, 'text'])

1120    Say hello to this unbelievably well behaved sq...
Name: text, dtype: object
1634    Two sneaky puppers were not initially seen, mo...
Name: text, dtype: object
189    @s8n You tried very hard to portray this good ...
Name: text, dtype: object
Series([], Name: text, dtype: object)
1779    IT'S PUPPERGEDDON. Total of 144/120 ...I think...
Name: text, dtype: object


In [23]:
#print whole text in order to verify numerators and denominators
print(data_twitter['text'][1120]) #17 dogs
print(data_twitter['text'][1634]) #13 dogs
print(data_twitter['text'][313]) #just a tweet to explain actual ratings, this will be ignored when cleaning data
print(data_twitter['text'][189]) #no picture, this will be ignored when cleaning data
print(data_twitter['text'][1779]) #12 dogs

Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq


In [24]:
data_twitter['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [25]:
print(data_twitter.loc[data_twitter.rating_denominator == 11, 'text']) 
print(data_twitter.loc[data_twitter.rating_denominator == 2, 'text']) 
print(data_twitter.loc[data_twitter.rating_denominator == 16, 'text']) 
print(data_twitter.loc[data_twitter.rating_denominator == 15, 'text'])
print(data_twitter.loc[data_twitter.rating_denominator == 7, 'text'])

784     RT @dog_rates: After so many requests, this is...
1068    After so many requests, this is Bretagne. She ...
1662    This is Darrel. He just robbed a 7/11 and is i...
Name: text, dtype: object
2335    This is an Albanian 3 1/2 legged  Episcopalian...
Name: text, dtype: object
1663    I'm aware that I could've said 20/16, but here...
Name: text, dtype: object
342    @docmisterio account started on 11/15/15
Name: text, dtype: object
516    Meet Sam. She smiles 24/7 &amp; secretly aspir...
Name: text, dtype: object


In [27]:
print(data_twitter['text'][784]) #retweet - it will be deleted when delete all retweets
print(data_twitter['text'][1068]) #actual rating 14/10 need to change manually
print(data_twitter['text'][1662]) #actual rating 10/10 need to change manually
print(data_twitter['text'][2335]) #actual rating 9/10 need to change manually
print(data_twitter['text'][1663]) # tweet to explain rating
print(data_twitter['text'][342]) #no rating - delete
print(data_twitter['text'][516]) #no rating - delete

RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ
This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5
This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv
I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible
@docmisterio account started on 11/15/15
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx


In [28]:
data_twitter['name'].value_counts()

None        745
a            55
Charlie      12
Lucy         11
Cooper       11
Oliver       11
Tucker       10
Lola         10
Penny        10
Bo            9
Winston       9
the           8
Sadie         8
Bailey        7
an            7
Daisy         7
Toby          7
Buddy         7
Jack          6
Oscar         6
Leo           6
Koda          6
Rusty         6
Dave          6
Stanley       6
Milo          6
Jax           6
Bella         6
Scout         6
Alfie         5
           ... 
Clybe         1
Lugan         1
Dutch         1
Finnegus      1
such          1
Samsom        1
Chef          1
Ember         1
Roscoe        1
Fabio         1
Shadoe        1
by            1
Tedders       1
Kara          1
Bobb          1
Ralphie       1
Holly         1
Lillie        1
Fiji          1
Mabel         1
Aiden         1
Kaia          1
Harnold       1
Pancake       1
Franq         1
Ito           1
Cal           1
Marq          1
Mookie        1
Trip          1
Name: name, Length: 957,

In [29]:
with pd.option_context('max_colwidth', 200):
    display(data_twitter[data_twitter['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

/Users/philipp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10


<a id="que"></a>
### Quality

##### Completeness, Validity, Accuracy, Consistency => a.k.a content issues

data_twitter
- [Keep original ratings (no retweets) that have images](#1que)
- [Some records have more than one dog stage)](#2que)
- [Convert timestamp column into datetime](#3que)
- [Correct numerators with decimals](#4que)
- [Correct denominators other than 10](#5que)
- [Erroneous dog names starting with lowercase characters (e.g. a, an, actually, by](#6que)

images
- [Drop 66 jpg_url duplicated](#7que)

favorite_retweet_table
- [Change tweet_id to type int64 in order to merge with the other 2 tables](#8que)


### Tidiness

##### Untidy data => a.k.a structural issues

data_twitter
- [Erroneous datatypes (doggo, floofer, pupper and puppo columns)](#1tid)

images
- [Create 1 column for image prediction and 1 column for confidence level](#2tid)

all tables
- [All tables should be part of one dataset](#3tid)

## <a id="cle">Cleaning Data</a>

In [39]:
data_twitter_clean = data_twitter.copy()
images_clean = images.copy()
json_clean = favorite_retweet_table.copy()

##### <a id="1que">1.Quality Issue - data_twitter:</a>
- Keep original ratings (no retweets) that have images

Based on info above, there are 181 values in retweeted_status_id and retweeted_status_user_id. Delete retweets. When I merge data_twitter with images, I will only take the ones with images.

In [40]:
#CODE: Delete retweets by filtering the NaN of retweeted_status_user_id
data_twitter_clean = data_twitter_clean[pd.isnull(data_twitter_clean['retweeted_status_user_id'])]

#TEST
print(sum(data_twitter_clean.retweeted_status_user_id.value_counts()))

0


##### <a id="2que">2.Quality Issue - data_twitter:</a>
- some records have more than one dog stage

In [41]:
print(len(data_twitter_clean[(data_twitter_clean.doggo != 'None') & (data_twitter_clean.floofer != 'None')]))
print(len(data_twitter_clean[(data_twitter_clean.doggo != 'None') & (data_twitter_clean.puppo != 'None')]))
print(len(data_twitter_clean[(data_twitter_clean.doggo != 'None') & (data_twitter_clean.pupper != 'None')]))

1
1
10


There is one record that has both doggo and floofer and another record that has both doggo and puppo. For these 2 records, take a look at the text manually to decide one dog stage for each of them. For ambiguous texts, set both the column values as None.
There are 10 records which have both doggo and pupper. As per the dogtionary, doggo and pupper are sometimes used interchangeably. Therefore, set pupper column as None for these 10 records.

In [42]:
#CODE
for i, row in data_twitter_clean[((data_twitter_clean.doggo != 'None') & (data_twitter_clean.floofer != 'None'))
                   | ((data_twitter_clean.doggo != 'None') & (data_twitter_clean.puppo != 'None'))].iterrows():
    print('%s %s\n'%(row.tweet_id, row.text))

855851453814013952 Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel

854010172552949760 At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk



In [43]:
# based on the above texts, doggo should be set as None for both the records
data_twitter_clean['doggo'][data_twitter_clean.tweet_id.isin([855851453814013952, 854010172552949760])] = 'None'

In [46]:
# set pupper column as None for records which have both doggo and pupper
data_twitter_clean['pupper'][(data_twitter_clean.doggo != 'None') & (data_twitter_clean.pupper != 'None')] = 'None'

In [47]:
# Test
len(data_twitter_clean[((data_twitter_clean.doggo != 'None') & (data_twitter_clean.pupper != 'None'))
                  | ((data_twitter_clean.doggo != 'None') & (data_twitter_clean.floofer != 'None')) 
                  | ((data_twitter_clean.doggo != 'None') & (data_twitter_clean.puppo != 'None'))])

0

##### <a id="3que">3.Quality Issue - data_twitter: </a>
- Convert timestamp column into datetime

In [50]:
#CODE: convert timestamp to datetime
data_twitter_clean['timestamp'] = pd.to_datetime(data_twitter_clean['timestamp'])

In [53]:
#TEST
data_twitter_clean['timestamp'].head()

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: timestamp, dtype: datetime64[ns]

##### <a id="4que">4.Quality Issue - data_twitter:</a>
- Correct numerators with decimals

In [54]:
data_twitter_clean[['rating_numerator', 'rating_denominator']] = data_twitter_clean[['rating_numerator','rating_denominator']].astype(float)

data_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2175 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2175 non-null datetime64[ns]
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null float64
rating_denominator            2175 non-null float64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
dtypes: datetime64[ns](1),

In [55]:
#CODE

#First change numerator and denominators type int to float to allow decimals 
data_twitter_clean[['rating_numerator', 'rating_denominator']] = data_twitter_clean[['rating_numerator','rating_denominator']].astype(float)

#Update numerators

data_twitter_clean.loc[(data_twitter_clean.tweet_id == 883482846933004288), 'rating_numerator'] = 13.5
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 786709082849828864), 'rating_numerator'] = 9.75
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 778027034220126208), 'rating_numerator'] = 11.27
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 681340665377193984), 'rating_numerator'] = 9.5
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 680494726643068929), 'rating_numerator'] = 11.26

#TEST
with pd.option_context('max_colwidth', 200):
    display(data_twitter_clean[data_twitter_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

/Users/philipp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,9.50,10.0
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0


##### <a id="5que">5.Quality Issue - data_twitter:</a>
- Correct denominators other than 10

Manually and programatically. Five tweets with denominator not equal to 10 for special circunstances. Update both numerators and denominators when necessary. Delete other five tweets because they do not have actual ratings.
These tweets with denominator not equal to 10 are multiple dogs.

In [56]:
#CODE: Update both numerators and denominators
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 740373189193256964), 'rating_numerator'] = 14
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 740373189193256964), 'rating_denominator'] = 10

data_twitter_clean.loc[(data_twitter_clean.tweet_id == 682962037429899265), 'rating_numerator'] = 10
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 682962037429899265), 'rating_denominator'] = 10

data_twitter_clean.loc[(data_twitter_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

data_twitter_clean.loc[(data_twitter_clean.tweet_id == 722974582966214656), 'rating_numerator'] = 13
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 722974582966214656), 'rating_denominator'] = 10

data_twitter_clean.loc[(data_twitter_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 13.5
data_twitter_clean.loc[(data_twitter_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10

#CODE: Delete five tweets with no actual ratings
data_twitter_clean = data_twitter_clean[data_twitter_clean['tweet_id'] != 832088576586297345]
data_twitter_clean = data_twitter_clean[data_twitter_clean['tweet_id'] != 810984652412424192]
data_twitter_clean = data_twitter_clean[data_twitter_clean['tweet_id'] != 682808988178739200]
data_twitter_clean = data_twitter_clean[data_twitter_clean['tweet_id'] != 835246439529840640]
data_twitter_clean = data_twitter_clean[data_twitter_clean['tweet_id'] != 686035780142297088]

#TEST: Left only the group dogs for programatically clean
with pd.option_context('max_colwidth', 200):
    display(data_twitter_clean[data_twitter_clean['rating_denominator'] != 10][['tweet_id',
                                                                                      'text',
                                                                                      'rating_numerator',
                                                                                      'rating_denominator']])

,tweet_id,text,rating_numerator,rating_denominator
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0,70.0
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0,150.0
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0,170.0
1228,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0,90.0
1254,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0,80.0
1274,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0,50.0
1351,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0,50.0
1433,697463031882764288,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44.0,40.0
1634,684225744407494656,"Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3",143.0,130.0
1635,684222868335505415,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121.0,110.0


In [57]:
#CODE: Create a new column with rating in float type to avoid converting all int column to float
data_twitter_clean['rating'] = 10 * data_twitter_clean['rating_numerator'] / data_twitter_clean['rating_denominator'].astype(float)

#TEST
data_twitter_clean.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
876,761004547850530816,NaN,NaN,2016-08-04 01:03:17,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo and Ty. Bo eats paper and Ty felt l...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/761004547...,11.0,10.0,Bo,None,None,None,None,11.0
115,870374049280663552,NaN,NaN,2017-06-01 20:18:38,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She really likes the planet. Wou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/870374049...,13.0,10.0,Zoey,None,None,None,None,13.0
443,819347104292290561,NaN,NaN,2017-01-12 00:55:47,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Anna and Elsa. They fall asleep i...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/819347104...,12.0,10.0,Anna,None,None,None,None,12.0
248,845397057150107648,NaN,NaN,2017-03-24 22:08:59,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Mimosa. She's an emotional suppor...,NaN,NaN,NaN,"https://www.gofundme.com/help-save-a-pup,https...",13.0,10.0,Mimosa,doggo,None,None,None,13.0
685,788150585577050112,NaN,NaN,2016-10-17 22:51:57,"<a href=""http://twitter.com/download/iphone"" r...",This is Leo. He's a golden chow. Rather h*ckin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/788150585...,13.0,10.0,Leo,None,None,None,None,13.0


##### <a id="6que">6.Quality Issue - data_twitter:</a>
- erroneous dog names starting with lowercase characters (e.g. a, an, actually, by)

Replace all lowercase values of name column with None

In [58]:
#CODE
data_twitter_clean['name'][data_twitter_clean['name'].str.match('[a-z]+')] = 'None'

/Users/philipp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [59]:
#TEST
data_twitter_clean.name[data_twitter_clean.name == 'None'].value_counts()

None    780
Name: name, dtype: int64

In [61]:
# Sort ascending by name to check if there are more names starting with a lowercase alphabet
data_twitter_clean.name.value_counts().sort_index(ascending=False)

Zuzu            1
Zooey           1
Zoey            3
Zoe             1
Ziva            1
Zeus            1
Zeke            3
Zeek            1
Zara            1
Yukon           1
Yogi            2
Yoda            1
Wyatt           3
Wishes          1
Winston         8
Winnie          4
Winifred        1
Wilson          3
Willy           1
Willow          1
Willie          1
William         1
Willem          1
Wiggles         1
Wesley          1
Watson          2
Walter          3
Wally           2
Wallace         3
Walker          1
               ..
Apollo          1
Antony          1
Anthony         1
Anna            1
Angel           1
Andy            1
Andru           1
Anakin          1
Amélie          1
Amy             1
Ambrose         1
Amber           1
Alice           2
Alfy            1
Alfie           4
Alf             1
Alexanderson    1
Alexander       1
Alejandro       1
Aldrick         1
Albus           2
Albert          2
Al              1
Akumi           1
Aja       

##### <a id="7que">7.Quality Issue - images:</a>
- Drop 66 jpg_url duplicated


In [62]:
#CODE: Delete duplicated jpg_url
images_clean = images_clean.drop_duplicates(subset=['jpg_url'], keep='last')

#TEST
sum(images_clean['jpg_url'].duplicated())

0

#####  <a id="8que">8.Quality Issue - favorite_retweet_table::</a>
- Change tweet_id to type int64 in order to merge with the other 2 tables

In [63]:
#CODE: change tweet_id from str to int
json_clean['tweet_id'] = json_clean['tweet_id'].astype(int)

#TEST
json_clean['tweet_id'].dtypes

dtype('int64')

##### <a id="1tid">1.Tidiness Issue - data_twitter:</a>
- Erroneous datatypes (doggo, floofer, pupper and puppo columns)

Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_stage column. Then drop dogs. Sort by dogs_stage in order to then drop duplicated based on tweet_id except for the last occurrence.

In [64]:
# Check the values in those columns by excuting those columns
print(data_twitter_clean.doggo.value_counts())
print(data_twitter_clean.floofer.value_counts())
print(data_twitter_clean.pupper.value_counts())
print(data_twitter_clean.puppo.value_counts())

None     2085
doggo      85
Name: doggo, dtype: int64
None       2160
floofer      10
Name: floofer, dtype: int64
None      1946
pupper     224
Name: pupper, dtype: int64
None     2145
puppo      25
Name: puppo, dtype: int64


In [65]:
#CODE
# Select the columns to melt and to remain
columns_to_melt = ['doggo', 'floofer', 'pupper', 'puppo']
columns_to_stay = [x for x in data_twitter_clean.columns.tolist() if x not in columns_to_melt]

# Melt the the columns into values
data_twitter_clean = pd.melt(data_twitter_clean, id_vars = columns_to_stay, value_vars = columns_to_melt, 
                         var_name = 'stages', value_name = 'dog_stage')

# Delete column 'stages'
data_twitter_clean = data_twitter_clean.drop('stages', 1)

# Filter for unique values then remove duplicate values based on 'dog_stage' values

#TEST 1
print(data_twitter_clean.dog_stage.value_counts())

data_twitter_clean = data_twitter_clean.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

#TEST 2
print(data_twitter_clean.dog_stage.value_counts())
print(len(data_twitter_clean))

None       8336
pupper      224
doggo        85
puppo        25
floofer      10
Name: dog_stage, dtype: int64
None       1826
pupper      224
doggo        85
puppo        25
floofer      10
Name: dog_stage, dtype: int64
2170


##### <a id="2tid">2.Tidiness Issue - images:</a>
- Create 1 column for image prediction and 1 column for confidence level

In [66]:
#CODE: the first true prediction (p1, p2 or p3) will be store in these lists
dog_type = []
confidence_list = []

#create a function with nested if to capture the dog type and confidence level
# from the first 'true' prediction
def sort_image(image_prediction):
    if image_prediction['p1_dog'] == True:
        dog_type.append(image_prediction['p1'])
        confidence_list.append(image_prediction['p1_conf'])
    elif image_prediction['p2_dog'] == True:
        dog_type.append(image_prediction['p2'])
        confidence_list.append(image_prediction['p2_conf'])
    elif image_prediction['p3_dog'] == True:
        dog_type.append(image_prediction['p3'])
        confidence_list.append(image_prediction['p3_conf'])
    else:
        dog_type.append('Error')
        confidence_list.append('Error')

#series objects having index the images_clean column.        
images_clean.apply(sort_image, axis=1)

#create new columns
images_clean['dog_type'] = dog_type
images_clean['confidence_list'] = confidence_list

#drop rows that has prediction_list 'error'
images_clean = images_clean[images_clean['dog_type'] != 'Error']

#TEST: 
images_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 2073
Data columns (total 14 columns):
tweet_id           1691 non-null int64
jpg_url            1691 non-null object
img_num            1691 non-null int64
p1                 1691 non-null object
p1_conf            1691 non-null float64
p1_dog             1691 non-null bool
p2                 1691 non-null object
p2_conf            1691 non-null float64
p2_dog             1691 non-null bool
p3                 1691 non-null object
p3_conf            1691 non-null float64
p3_dog             1691 non-null bool
dog_type           1691 non-null object
confidence_list    1691 non-null object
dtypes: bool(3), float64(3), int64(2), object(6)
memory usage: 163.5+ KB


##### <a id="3tid">3.Tidiness Issue - All tables:</a>

- merge all tables to one dataset

In [67]:
#Delete columns which are not needed for the analysis
#get the column names of data_twitter_clean
print(list(data_twitter_clean))


#CODE: Delete columns no needed
data_twitter_clean = data_twitter_clean.drop(['source',
                                                    'in_reply_to_status_id',
                                                    'in_reply_to_user_id',
                                                    'retweeted_status_id',
                                                    'retweeted_status_user_id', 
                                                    'retweeted_status_timestamp', 
                                                    'expanded_urls'], 1)

['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'rating', 'dog_stage']


In [68]:
#CODE: print list of image_prediction columns
print(list(images_clean))

#Delete columns
images_clean = images_clean.drop(['img_num', 'p1', 
                                                      'p1_conf', 'p1_dog', 
                                                      'p2', 'p2_conf', 
                                                      'p2_dog', 'p3', 
                                                      'p3_conf', 
                                                      'p3_dog'], 1)

#TEST
list(images_clean)

['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'dog_type', 'confidence_list']


['tweet_id', 'jpg_url', 'dog_type', 'confidence_list']

In [69]:
#CODE: create a new dataframe that merge data_twitter_clean and images_clean
df_twitter = pd.merge(data_twitter_clean, 
                      images_clean, 
                      how = 'left', on = ['tweet_id'])

#keep rows that have picture (jpg_url)
df_twitter = df_twitter[df_twitter['jpg_url'].notnull()]

#TEST
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 0 to 2169
Data columns (total 11 columns):
tweet_id              1625 non-null int64
timestamp             1625 non-null datetime64[ns]
text                  1625 non-null object
rating_numerator      1625 non-null float64
rating_denominator    1625 non-null float64
name                  1625 non-null object
rating                1625 non-null float64
dog_stage             1625 non-null object
jpg_url               1625 non-null object
dog_type              1625 non-null object
confidence_list       1625 non-null object
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 152.3+ KB


In [70]:
#CODE: create a new dataframe that merge df_twitter and json_clean
df_twitter_master = pd.merge(df_twitter, json_clean, 
                      how = 'left', on = ['tweet_id'])

#TEST
df_twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 0 to 1624
Data columns (total 13 columns):
tweet_id              1625 non-null int64
timestamp             1625 non-null datetime64[ns]
text                  1625 non-null object
rating_numerator      1625 non-null float64
rating_denominator    1625 non-null float64
name                  1625 non-null object
rating                1625 non-null float64
dog_stage             1625 non-null object
jpg_url               1625 non-null object
dog_type              1625 non-null object
confidence_list       1625 non-null object
favorite_count        1624 non-null float64
retweet_count         1624 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(1), object(6)
memory usage: 177.7+ KB


In [71]:
df_twitter_master.to_csv('twitter_archive_master.csv', index=False)